### To do
1. Calculate weights for the 12 level series

In [23]:
# import required directories
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# read raw data
df = pd.read_csv('../data/raw/sales_train_validation.csv')
price_df = pd.read_csv('../data/raw/sell_prices.csv')
cal_df = pd.read_csv('../data/raw/calendar.csv')

In [3]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [4]:
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [5]:
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [6]:
# data preprocessing
# Removing 'd' from the days columns
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))
cal_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1


In [7]:
# data preprocessing
# creating 'id' column in price_df which is a combination of item_id + store_id + "valuation" so as to make a match with the id column of df
price_df['id'] = price_df['item_id'] + '_' + price_df['store_id'] +  '_validation'
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price,id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES_1_001_CA_1_validation
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES_1_001_CA_1_validation
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES_1_001_CA_1_validation
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES_1_001_CA_1_validation
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES_1_001_CA_1_validation


In [8]:
df.shape

(30490, 1919)

In [9]:
price_df.shape

(6841121, 5)

In [10]:
cal_df.shape

(1969, 14)

#### Train - Validation - Test data set
* From the dataset given to us, we take the last 28 days as the test set and the 28 days prior to this as validataion set. Remaining data is used for training set.
* In the sales_train_validation.csv file, we have data for 1 to 1913 days. 
* Last 28 days are for test data. So days from 1886 to 1913 are for testing. (1886 = 1913 - 28)
* Prior to these 28 days, we take last 28 days as validation set. So days from 1858 to 1885 are for validation set. (1858 = 1885 - 28)
* Rest of the days from day 1 to day 1857 for training data.

In [24]:
# validation data
for day in tqdm(range(1858, 1886)):
    wk_id = list(cal_df[cal_df["d"]==day]["wm_yr_wk"])[0]
    wk_price_df = price_df[price_df["wm_yr_wk"]==wk_id]
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:30<00:00,  1.08s/it]


In [25]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,unit_sales_1876,unit_sales_1877,unit_sales_1878,unit_sales_1879,unit_sales_1880,unit_sales_1881,unit_sales_1882,unit_sales_1883,unit_sales_1884,unit_sales_1885
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,24.78,8.26,24.78,8.26,16.52,16.52,0.00,8.26,8.26,8.26
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,3.97,3.97,3.97,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.97,0.00,0.00,0.00,0.00,0.00,0.00,2.97,2.97,0.00
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,18.56,9.28,4.64,18.56,4.64,13.92,23.20,0.00,27.84,27.84
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8.64,5.76,5.76,5.76,8.64,2.88,0.00,0.00,0.00,0.00


In [26]:
# Creating a row-wise sum for unit sales for validation
df['dollar_sales'] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1)
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,unit_sales_1877,unit_sales_1878,unit_sales_1879,unit_sales_1880,unit_sales_1881,unit_sales_1882,unit_sales_1883,unit_sales_1884,unit_sales_1885,dollar_sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8.26,24.78,8.26,16.52,16.52,0.00,8.26,8.26,8.26,231.28
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,3.97,3.97,3.97,3.97,35.73
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,2.97,2.97,0.00,44.55
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,9.28,4.64,18.56,4.64,13.92,23.20,0.00,27.84,27.84,245.92
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5.76,5.76,5.76,8.64,2.88,0.00,0.00,0.00,0.00,89.28


In [27]:
# dropping unit sales column from the validation set
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True)
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,231.28
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,35.73
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,44.55
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,245.92
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,89.28


In [28]:
# Calculating weight 
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,dollar_sales,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,231.28,0.000063
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,35.73,0.000010
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,44.55,0.000012
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,245.92,0.000067
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,89.28,0.000024


In [29]:
df.drop(columns=["dollar_sales"], inplace=True)
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,0.000063
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.000010
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,0.000012
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,0.000067
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,0.000024


In [30]:
# Creating columns for predictions
for d in range(1886, 1914):
    df["F_" + str(d)] = 0

In [31]:
df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,0,0,0,0,0,0,0,0,0,0
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,0,0,0,0,0,0,0,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30489,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [33]:
agg_df.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,0,0,0,0,0,0,0,0,1,0.083333


In [34]:
level_groupings = {2: ["state_id"], 
                   3: ["store_id"], 
                   4: ["cat_id"], 
                   5: ["dept_id"],
                   6: ["state_id", "cat_id"], 
                   7: ["state_id", "dept_id"], 
                   8: ["store_id", "cat_id"], 
                   9: ["store_id", "dept_id"],
                   10: ["item_id"], 
                   11: ["item_id", "state_id"]}

In [35]:
df.groupby(by=level_groupings[11]).sum()

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         state_id                                                      
FOODS_1_001     CA          6    3    2    3    7    5    8    3    5     2   
                TX          0    1    2    2    0    4    0    0    4     3   
                WI          0    2    0    1    0    9    2    1    2     5   
FOODS_1_002     CA          3    3    4    4    3    3    0    2    1     1   
                TX          0    0    2    0    0    0    0    1    0     0   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_515 TX          0    0    0    0    0    0    0    0    0     0   
                WI          0    0    0    0    0    0    0    0    0     0   
HOUSEHOLD_2_516 CA          0    2    0    1    1    1    0    2    1     2   
                TX          2    1    0    0    0    0    1    1    0     0   
                WI          0    1    0    2    0    1    1    1    0     0   

                          ...  F_1904  F_1905  F_1906  F_1907  F_1908  F_1909  \
item_id         state_id  ...                                                   
FOODS_1_001     CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   
FOODS_1_002     CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_515 TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   
HOUSEHOLD_2_516 CA        ...       0       0       0       0       0       0   
                TX        ...       0       0       0       0       0       0   
                WI        ...       0       0       0       0       0       0   

                          F_1910  F_1911  F_1912  F_1913  
item_id         state_id                                  
FOODS_1_001     CA             0       0       0       0  
                TX             0       0       0       0  
                WI             0       0       0       0  
FOODS_1_002     CA             0       0       0       0  
                TX             0       0       0       0  
...                          ...     ...     ...     ...  
HOUSEHOLD_2_515 TX             0       0       0       0  
                WI             0       0       0       0  
HOUSEHOLD_2_516 CA             0       0       0       0  
                TX             0       0       0       0  
                WI             0       0       0       0  

[9147 rows x 1942 columns]

In [36]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df["level"] = level
    temp_df["weight"] /= 12
    agg_df = agg_df.append(temp_df[column_order])

del temp_df
agg_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:31<00:00,  3.18s/it]


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913,level,weight
0,32631,31749,23783,25412,19146,29211,28010,37932,32736,25572,...,0,0,0,0,0,0,0,0,1,0.083333
0,14195,13805,10108,11047,9925,11322,12251,16610,14696,11822,...,0,0,0,0,0,0,0,0,2,0.036146
1,9438,9630,6778,7381,5912,9006,6226,9440,9376,7319,...,0,0,0,0,0,0,0,0,2,0.023281
2,8998,8314,6897,6984,3309,8883,9533,11882,8664,6431,...,0,0,0,0,0,0,0,0,2,0.023907
0,4337,4155,2816,3051,2630,3276,3450,5437,4340,3157,...,0,0,0,0,0,0,0,0,3,0.009092


In [38]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df["weight"] /= 12
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1904,F_1905,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])

30490 12350 42840


In [41]:
agg_df["weight"].sum() + df["weight"].sum()

0.9999999999999996

### Calculalte RMSSE for all series using the equation

In [42]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [43]:
train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]  # everything except last 28
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]  # final 28 
forecast_cols = [c for c in df.columns if c.find("F_") == 0]

In [45]:
train_series_cols[-9:]

['d_1877',
 'd_1878',
 'd_1879',
 'd_1880',
 'd_1881',
 'd_1882',
 'd_1883',
 'd_1884',
 'd_1885']

In [46]:
ground_truth_cols[-9:]

['d_1905',
 'd_1906',
 'd_1907',
 'd_1908',
 'd_1909',
 'd_1910',
 'd_1911',
 'd_1912',
 'd_1913']

In [47]:
forecast_cols

['F_1886',
 'F_1887',
 'F_1888',
 'F_1889',
 'F_1890',
 'F_1891',
 'F_1892',
 'F_1893',
 'F_1894',
 'F_1895',
 'F_1896',
 'F_1897',
 'F_1898',
 'F_1899',
 'F_1900',
 'F_1901',
 'F_1902',
 'F_1903',
 'F_1904',
 'F_1905',
 'F_1906',
 'F_1907',
 'F_1908',
 'F_1909',
 'F_1910',
 'F_1911',
 'F_1912',
 'F_1913']

In [48]:
df["rmsse"] = rmsse(np.array(df[ground_truth_cols]), 
                   np.array(df[forecast_cols]), np.array(df[train_series_cols]))
agg_df["rmsse"] = rmsse(np.array(agg_df[ground_truth_cols]), 
                   np.array(agg_df[forecast_cols]), np.array(agg_df[train_series_cols]))

In [49]:
df["wrmsse"] = df["weight"] * df["rmsse"]
agg_df["wrmsse"] = agg_df["weight"] * agg_df["rmsse"]

In [50]:
df["wrmsse"].sum() + agg_df["wrmsse"].sum()

5.35979273095658

### Use the naive logic to make forecasts for each of the level 12 series
* Use the naive logic to make forecasts for each of the level 12 series
* All 0s
* Average through all history
* Mean of previous 10, 20, 30, 40, 50, 60 days
* Same as previous 28 days
* Average of same day for all previous weeks

In [51]:
# Mean of history
df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) <= 1885] +\
       ["id"]].set_index("id").transpose()

id,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,HOBBIES_1_009_CA_1_validation,HOBBIES_1_010_CA_1_validation,...,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0
d_3,0,0,0,0,0,0,0,0,7,1,...,0,5,1,0,2,2,0,0,0,0
d_4,0,0,0,0,0,0,0,0,3,0,...,0,6,1,0,5,2,0,2,0,0
d_5,0,0,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1881,2,0,0,3,1,0,0,10,0,0,...,2,4,1,0,2,0,0,0,2,5
d_1882,0,1,0,5,0,3,1,2,0,0,...,0,2,2,0,3,0,0,0,1,3
d_1883,1,1,1,0,0,1,0,0,0,0,...,6,2,3,0,1,0,0,2,1,2
d_1884,1,1,1,6,0,1,0,0,0,0,...,2,1,2,2,1,0,0,0,2,0


In [52]:
complete_historical_mean_df =\
    df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) <= 1885] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

In [53]:
complete_historical_mean_df.head()

,id,0
0,HOBBIES_1_001_CA_1_validation,0.303979
1,HOBBIES_1_002_CA_1_validation,0.260477
2,HOBBIES_1_003_CA_1_validation,0.144297
3,HOBBIES_1_004_CA_1_validation,1.717241
4,HOBBIES_1_005_CA_1_validation,0.960743


In [54]:
# Nothing is always 0
df[[c for c in df.columns if c.find("d_")==0]].sum(axis=1).min()

10

In [55]:
def find_first_non_0(s):
    assert type(s) == np.ndarray
    return (s!=0).argmax(axis=0)

In [60]:
non_0_strt_arr = []
hist_arr = np.array(df[[c for c in df.columns if c.find("d_")==0]])
for i in tqdm(range(len(df))):
    non_0_strt_arr.append(find_first_non_0(hist_arr[i, :]))

100%|█████████████████████████████████████████████████████████████████████████| 30490/30490 [00:02<00:00, 12785.69it/s]


In [61]:
df.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1906,F_1907,F_1908,F_1909,F_1910,F_1911,F_1912,F_1913,rmsse,wrmsse
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1.768048,0.000009


In [62]:
test = list(df[[c for c in df.columns if c.find("d_")==0] +\
                ["id"]].set_index("id").transpose()["HOBBIES_1_001_CA_1_validation"])

In [63]:
print("Supposedly first non-zero value equals:", test[non_0_strt_arr[0]], 
      "on the", non_0_strt_arr[0], "day",
     "\nSum of all values before the supposedly first non-zero value is:", 
     sum(test[: non_0_strt_arr[0]]),
     "\nSum of all values after the supposedly first non-zero value is:", 
     sum(test[non_0_strt_arr[0]:]))

Supposedly first non-zero value equals: 1 on the 901 day 
Sum of all values before the supposedly first non-zero value is: 0 
Sum of all values after the supposedly first non-zero value is: 600


In [64]:
num_non_zero = 1885 - np.array(non_0_strt_arr)

In [65]:
non_zero_historical_mean_arr = np.array(df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) <= 1885] +\
   ["id"]].set_index("id").transpose().sum().reset_index()[0]) / num_non_zero

In [66]:
# days 1886 to 1913 are local test weeks
for d in range(1, 29):
    df["F_1_" + str(1885+d)] = list(complete_historical_mean_df[0])
    df["F_2_" + str(1885+d)] = non_zero_historical_mean_arr

In [71]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1_1909,F_2_1909,F_1_1910,F_2_1910,F_1_1911,F_2_1911,F_1_1912,F_2_1912,F_1_1913,F_2_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.303979,0.582317,0.303979,0.582317,0.303979,0.582317,0.303979,0.582317,0.303979,0.582317
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.260477,0.281860,0.260477,0.281860,0.260477,0.281860,0.260477,0.281860,0.260477,0.281860
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.144297,0.348718,0.144297,0.348718,0.144297,0.348718,0.144297,0.348718,0.144297,0.348718
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.717241,1.750676,1.717241,1.750676,1.717241,1.750676,1.717241,1.750676,1.717241,1.750676
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.960743,1.021433,0.960743,1.021433,0.960743,1.021433,0.960743,1.021433,0.960743,1.021433


In [67]:
method_dict = {1: "complete historical mean", 2: "historical mean after first non-zero"}

#### Mean of recent x days

In [68]:
num_non_zero.min()

40

In [69]:
historical_mean_df10 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1876, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

historical_mean_df20 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1866, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

historical_mean_df30 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1856, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

historical_mean_df40 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1846, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

In [70]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_1_1909,F_2_1909,F_1_1910,F_2_1910,F_1_1911,F_2_1911,F_1_1912,F_2_1912,F_1_1913,F_2_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.303979,0.582317,0.303979,0.582317,0.303979,0.582317,0.303979,0.582317,0.303979,0.582317
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.260477,0.281860,0.260477,0.281860,0.260477,0.281860,0.260477,0.281860,0.260477,0.281860
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.144297,0.348718,0.144297,0.348718,0.144297,0.348718,0.144297,0.348718,0.144297,0.348718
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.717241,1.750676,1.717241,1.750676,1.717241,1.750676,1.717241,1.750676,1.717241,1.750676
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.960743,1.021433,0.960743,1.021433,0.960743,1.021433,0.960743,1.021433,0.960743,1.021433


In [72]:
# days 1886 to 1913 are local test weeks
for d in range(1, 29):
    df["F_3_" + str(1885+d)] = list(historical_mean_df10[0])
    df["F_4_" + str(1885+d)] = list(historical_mean_df20[0])
    df["F_5_" + str(1885+d)] = list(historical_mean_df30[0])
    df["F_6_" + str(1885+d)] = list(historical_mean_df40[0])

df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_5_1911,F_6_1911,F_3_1912,F_4_1912,F_5_1912,F_6_1912,F_3_1913,F_4_1913,F_5_1913,F_6_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.133333,1.100,1.5,1.15,1.133333,1.100,1.5,1.15,1.133333,1.100
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.300000,0.275,0.4,0.30,0.300000,0.275,0.4,0.30,0.300000,0.275
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.500000,0.450,0.3,0.35,0.500000,0.450,0.3,0.35,0.500000,0.450
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.966667,1.975,3.2,2.15,1.966667,1.975,3.2,2.15,1.966667,1.975
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1.100000,1.025,1.3,1.20,1.100000,1.025,1.3,1.20,1.100000,1.025


In [73]:
method_dict[3] = "historical mean of recent 10 days"
method_dict[4] = "historical mean of recent 20 days"
method_dict[5] = "historical mean of recent 30 days"
method_dict[6] = "historical mean of recent 40 days"

In [74]:
#### Same as previous 28 days
for d in range(1, 29):
    df["F_7_" + str(1885 + d)] = df["d_" + str(1885 + d - 28)]

In [75]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_7_1904,F_7_1905,F_7_1906,F_7_1907,F_7_1908,F_7_1909,F_7_1910,F_7_1911,F_7_1912,F_7_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,1,3,1,2,2,0,1,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,4,2,1,4,1,3,5,0,6,6
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,2,2,2,3,1,0,0,0,0


In [76]:
method_dict[7] = "same as last 28 days"

In [77]:
#### Historical averages from same day of the year
denominator = [(num // 364) if (num // 364) > 0 else 1 for num in num_non_zero]
for d in range(1, 29): 
    df["F_8_" + str(1885 + d)] = (df["d_" + str(1885 + d - 364*1)]+\
                                  df["d_" + str(1885 + d - 364*2)]+\
                                  df["d_" + str(1885 + d - 364*3)]+\
                                  df["d_" + str(1885 + d - 364*4)]+\
                     df["d_" + str(1885 + d - 364*5)]) / denominator
    
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,F_8_1904,F_8_1905,F_8_1906,F_8_1907,F_8_1908,F_8_1909,F_8_1910,F_8_1911,F_8_1912,F_8_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.50,1.50,0.00,1.0,0.0,1.00,0.50,0.00,1.00,0.50
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.25,0.25,0.00,0.0,0.5,0.25,0.25,0.25,0.00,0.00
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.00,0.50,0.00,0.0,0.0,0.00,0.50,0.50,0.00,0.00
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2.00,3.40,2.20,1.6,1.4,1.20,2.00,1.60,1.40,2.60
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0.75,1.50,2.25,0.0,0.5,0.75,0.75,0.50,0.75,0.75


In [78]:
method_dict[8] = "average of same day in historical years"

In [79]:
agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [80]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_8_1906,F_8_1907,F_8_1908,F_8_1909,F_8_1910,F_8_1911,F_8_1912,F_8_1913,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,45755.466667,35363.383333,32867.816667,32691.55,32935.033333,37391.55,46296.35,47663.15,1,0.083333


In [83]:
train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]

forecast_cols_dict = {}
for i in range(1, 9):
    forecast_cols_dict[i] = [c for c in df.columns if c.find("F_"+str(i)+"_") == 0]

In [84]:
for i in range(1, 9):
    df["rmsse_" + str(i)] = rmsse(np.array(df[ground_truth_cols]), 
        np.array(df[forecast_cols_dict[i]]), np.array(df[train_series_cols]))
    agg_df["rmsse_" + str(i)] = rmsse(np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[forecast_cols_dict[i]]), np.array(agg_df[train_series_cols]))

In [95]:
for i in range(1, 9):
    df["wrmsse_" + str(i)] = df["weight"] * df["rmsse_" + str(i)] * 0.002
    agg_df["wrmsse_" + str(i)] = agg_df["weight"] * agg_df["rmsse_" + str(i)] * 0.002

In [96]:
for i in range(1, 9):
    print("method:", method_dict[i])
    print(df["wrmsse_" + str(i)].sum() + agg_df["wrmsse_" + str(i)].sum())
    print()

method: complete historical mean
0.0005065315260809282

method: historical mean after first non-zero
0.0003913033927030908

method: historical mean of recent 10 days
0.00037696334211064284

method: historical mean of recent 20 days
0.0003741155714817119

method: historical mean of recent 30 days
0.0003737539233891865

method: historical mean of recent 40 days
0.00037344042971149946

method: same as last 28 days
0.00031910904110208564

method: average of same day in historical years
0.0003044904345512866



In [97]:
sample_sub = pd.read_csv("../data/raw/sample_submission.csv")

In [98]:
sample_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
submit_df = df[["id"]]
for i in range(1, 29):
    submit_df["F" + str(i)] = df["F_7_" + str(1885 + i)]

C:\Users\z0045dra\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [100]:
submit_df2 = submit_df.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [101]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)

In [102]:
submit_df.to_csv("../submissions/submission_naiveforecasts_002_04102020.csv", index=False)